In [1]:
import os
import numpy as np
from nanowire.optics.simulate import Simulator
from nanowire.optics.postprocess import Simulation
from nanowire.optics.utils.utils import setup_sim
from nanowire.optics.utils.config import Config
import scipy.constants as consts
import scipy.integrate as intg
import IPython.display as disp
import matplotlib.pyplot as plt
%load_ext autoreload 
%autoreload 2

In [2]:
conf = Config('AbsorptionTest.yml')
sim = Simulator(conf)
sim = setup_sim(sim)
Zo = consts.physical_constants['characteristic impedance of vacuum'][0]

In [3]:
fluxes = sim.get_fluxes()

In [4]:
total_incident_power = .5*sim.period**2/Zo*np.absolute(fluxes['Air'][0])
total_reflected_power = .5*sim.period**2/Zo*np.absolute(fluxes['Air'][1])
total_transmitted_power = .5*sim.period**2/Zo*np.absolute(sum(fluxes['Substrate_bottom']))
total_absorbed_power = total_incident_power - total_reflected_power - total_transmitted_power
print('Total Incident Power = {}'.format(total_incident_power))
print('Total Reflected Power = {}'.format(total_reflected_power))
print('Total Transmitted Power = {}'.format(total_transmitted_power))
print('Total Absorbed Power = {}'.format(total_absorbed_power))

Total Incident Power = 0.00016590117058987953
Total Reflected Power = 1.290540089566545e-05
Total Transmitted Power = 7.410460196401563e-06
Total Absorbed Power = 0.00014558530949781252


In [5]:
summed_absorbed_power = 0
abs_dict_fluxmethod = {}
for layer, (forw_top, back_top) in fluxes.items():
    if '_bottom' in layer:
        continue
    bottom = layer+'_bottom'
    forw_bot, back_bot = fluxes[bottom] 
    print('-'*25)
    print('Layer: {}'.format(layer))
    print('Forward Top: {}'.format(forw_top))
    print('Backward Top: {}'.format(back_top))
    print('Forward Bottom: {}'.format(forw_bot))
    print('Backward Bottom: {}'.format(back_bot))
    P_in = forw_top + -1*back_bot
    P_out = forw_bot + -1*back_top
    print('Power Entering Layer: {}'.format(P_in))
    print('Power Leaving Layer: {}'.format(P_out))
    P_lost = P_in - P_out
    P_abs = .5*P_lost*(sim.period**2)/Zo
    abs_dict_fluxmethod[layer] = P_abs 
    print('Absorbed in Layer: {}'.format(P_abs))
    summed_absorbed_power += P_abs
print('-'*25)
print('Summed Absorption= {}'.format(summed_absorbed_power))

-------------------------
Layer: NW_SiO2
Forward Top: (0.6415681690545958-0.002683480964878013j)
Backward Top: (-0.054474731321943956+0.002683480964878013j)
Forward Bottom: (0.632688624556471-0.0003835546834251788j)
Backward Bottom: (-0.05485000039465573+0.0003835546834251788j)
Power Entering Layer: (0.6964181694492515-0.0030670356483031916j)
Power Leaving Layer: (0.6871633558784149-0.0030670356483031916j)
Absorbed in Layer: (7.676922024964452e-07+0j)
-------------------------
Layer: Air
Forward Top: (2+0j)
Backward Top: (-0.15557938319276354+0j)
Forward Bottom: (2+0j)
Backward Bottom: (-0.15557938319276352+0j)
Power Entering Layer: (2.1555793831927637+0j)
Power Leaving Layer: (2.1555793831927637+0j)
Absorbed in Layer: 0j
-------------------------
Layer: ITO
Forward Top: (1.911743947569879+0.0008713251712626491j)
Backward Top: (-0.06732333076256294-0.0008713251712626491j)
Forward Bottom: (1.7901404153159508-0.0023688997496048164j)
Backward Bottom: (-0.06949267936052105+0.00236889974960

In [6]:
def integrate(arr, layer_obj, sim_proc):
    arr_slice = arr[layer_obj.slice]
    zsamps = layer_obj.iend - layer_obj.istart
    z_vals = np.linspace(0, layer_obj.thickness, zsamps)
    x_vals = np.linspace(0, sim_proc.period, sim_proc.x_samples)
    y_vals = np.linspace(0, sim_proc.period, sim_proc.y_samples)
    z_integral = intg.trapz(arr_slice, x=z_vals, axis=0)
    x_integral = intg.trapz(z_integral, x=x_vals, axis=0)
    y_integral = intg.trapz(x_integral, x=y_vals, axis=0)
    return y_integral

In [7]:
sim.get_field()

In [8]:
sim_proc = Simulation(simulator=sim)
freq = sim_proc.conf[('Simulation', 'params', 'frequency','value')]
try:
    Esq = sim_proc.data['normEsquared']
except KeyError:
    Esq = sim_proc.normEsquared()
abs_dict_intmethod = {}
for layer_name, layer_obj in sim_proc.layers.items():
    print("Layer: {}".format(layer_name))
    base_unit = sim_proc.conf[('Simulation', 'base_unit')]
    n_mat, k_mat = layer_obj.get_nk_matrix(si_freq)
    # n and k could be functions of space, so we need to multiply the
    # fields by n and k before integrating
    res = integrate(Esq*n_mat*k_mat,layer_obj, sim_proc)
    p_abs_imag = 2*np.pi*freq*consts.epsilon_0*res*base_unit
    abs_dict_intmethod[layer_name] = p_abs_imag
    disp.display_latex("$P_{abs} = \\frac{\omega}{2} Im(\epsilon) \int |E|^2 dV"+" = {}$".format(p_abs_imag), raw=True)

Layer: Air


NameError: name 'si_freq' is not defined

In [ ]:
print(abs_dict_fluxmethod)
print(abs_dict_intmethod)

In [ ]:
for key in abs_dict_fluxmethod.keys():
    fm = abs_dict_fluxmethod[key]
    im = abs_dict_intmethod[key]
    diff = fm - im
    try:
        pdiff = 100*abs(diff)/fm
    except ZeroDivisionError:
        pdiff = None
        pass
    print('-'*25)
    print("Layer: {}".format(key))
    print("Flux Method: {}".format(fm))
    print("Integral Method: {}".format(im))
    print("Diff: {}".format(diff))
    print("Percent Diff: {}".format(pdiff))